In [2]:
import sys
# sys.path.append('/Users/bernardoloureiro/template-lib')

from utils.notebookhelpers.helpers import Helpers
from utils.dtos.templateOutputCollection import TemplateOutputCollection
from utils.dtos.variable import Metadata
from utils.dtos.templateOutput import TemplateOutput
from utils.dtos.templateOutput import OutputType
from utils.dtos.templateOutput import ChartType
import datetime
import logging
import pandas as pd
import numpy as np
import os
from dateutil import parser
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
import math
import scipy as scipy
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

inputDatasetParameter=Helpers.get_or_create_input_dataset(
    name="inputDataset",
    metadata=Metadata(input_name='Input Dataset', is_required=True, tooltip='Dataset to apply the transformation'),
    local_context=locals()
)
start_dateParameter = Helpers.get_or_create_input_var(
    name="train_bene_df",
    metadata=Metadata(input_name="train_bene_df", is_required=True, tooltip="Benefeciary", multiple=False, datatypes=['TIMESTAMP'], options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']),
    local_context=locals()
)
end_dateParameter = Helpers.get_or_create_input_var(
    name="train_ip_df",
    metadata=Metadata(input_name="train_ip_df", is_required=True, tooltip="InPateint data", multiple=False, datatypes=['TIMESTAMP'], options=['FIELDS', 'CONSTANT'], datasets=['inputDataset']
),
    local_context=locals()
)
howParameter = Helpers.get_or_create_input_var(
    name="train_op_df",
    metadata=Metadata(input_name="train_op_df", is_required=True, tooltip="OutPataint data",default_value='seconds', multiple=False, datatypes=['STRING'], options=['CONSTANT'], constant_options=['miliseconds', 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', 'years']
),
    local_context=locals()
)
outputcolumnParameter = Helpers.get_or_create_input_var(
    name="train_tgt_lbls_df",
    metadata=Metadata(input_name="train_tgt_lbls_df", is_required=True, tooltip="Train Target label", multiple=False, datatypes=['STRING'], options=['CONSTANT']
),
    local_context=locals()
)
outputDatasetParameter=Helpers.get_or_create_output_dataset(
 name="outputDataset",
    metadata=Metadata(input_name='Output Dataset', is_required=True, tooltip='Dataset name to be created after the transformation'),
    local_context=locals()
)

In [3]:
inputDatasetParameter=Helpers.get_or_create_input_dataset(
    name="inputDataset",
    metadata=Metadata(input_name='train_bene_df', is_required=True, tooltip='Benefeciary Dataset to apply the transformation'),
    local_context=locals()
)

inputDatasetParameter2=Helpers.get_or_create_input_dataset(
    name="inputDataset2",
    metadata=Metadata(input_name='train_ip_df', is_required=True, tooltip='InPateint Dataset to apply the transformation'),
    local_context=locals()
)

inputDatasetParameter3=Helpers.get_or_create_input_dataset(
    name="inputDataset3",
    metadata=Metadata(input_name='train_op_df', is_required=True, tooltip='OpPateint Dataset to apply the transformation'),
    local_context=locals()
)

inputDatasetParameter4=Helpers.get_or_create_input_dataset(
    name="inputDataset4",
    metadata=Metadata(input_name='train_tgt_lbls_df', is_required=True, tooltip='Target Dataset to apply the transformation'),
    local_context=locals()
)



outputDatasetParameter=Helpers.get_or_create_output_dataset(
 name="outputDataset",
    metadata=Metadata(input_name='Output Dataset', is_required=True, tooltip='Dataset name to be created after the transformation'),
    local_context=locals()
)

In [4]:
#Getting the context
contextId = 'HealtCare_Fraud'
context = Helpers.getOrCreateContext(contextId=contextId, localVars=locals())

#Getting the parameters
inputDataset=inputDatasetParameter.value
start_date=start_dateParameter.value
end_date=end_dateParameter.value
how=howParameter.value
outputcolumn=outputcolumnParameter.value
outputDataset=outputDatasetParameter.value

In [10]:
#Getting the parameters
train_bene_df=inputDatasetParameter.value
train_ip_df= inputDatasetParameter2.value
train_op_df= inputDatasetParameter3.value
train_tgt_lbls_df= inputDatasetParameter4.value



outputDataset=outputDatasetParameter.value

In [6]:
train_bene_df = Helpers.getEntityData(context, train_bene_df)
train_ip_df = Helpers.getEntityData(context, train_ip_df)
train_op_df = Helpers.getEntityData(context, train_op_df)
train_tgt_lbls_df = Helpers.getEntityData(context, train_tgt_lbls_df)

entity doesn't exist
entity doesn't exist
entity doesn't exist
entity doesn't exist


In [7]:
def feature_Enginerring(train_bene_df,train_ip_df,train_op_df,train_tgt_lbls_df):

    train_ip_df["Admitted?"] = 1
    train_op_df["Admitted?"] = 0
    # Merging the IP and OP dataset on the basis of common columns.
    common_cols = [col for col in train_ip_df.columns if col in train_op_df.columns]
    train_ip_op_df = pd.merge(left=train_ip_df, right=train_op_df, left_on=common_cols, right_on=common_cols, how="outer")
    train_ip_op_bene_df = pd.merge(left=train_ip_op_df, right=train_bene_df, left_on='BeneID', right_on='BeneID',how='inner')
    ### **Merging the IP_OP_BENE Dataset with PROVIDER level Tgt Labels Data**
    train_iobp_df = pd.merge(left=train_ip_op_bene_df, right=train_tgt_lbls_df, left_on='Provider', right_on='Provider',how='inner')

    # Joining with the PRV Tgt Labels
    prvs_claims_df = pd.DataFrame(train_iobp_df.groupby(['Provider'])['ClaimID'].count()).reset_index()
    prvs_claims_tgt_lbls_df = pd.merge(left=prvs_claims_df, right=train_tgt_lbls_df, on='Provider', how='inner')


    #Feature-2
    ## Is Alive? = No if DOD is NaN else Yes

    train_iobp_df['DOB'] = pd.to_datetime(train_iobp_df['DOB'], format="%Y-%m-%d")
    train_iobp_df['DOD'] = pd.to_datetime(train_iobp_df['DOD'], format="%Y-%m-%d")
#     train_iobp_df['Is_Alive?'] = train_iobp_df['DOD'].apply(lambda val: 'No' if val != val else 'Yes')

    ## Adding New Feature - 3 :: Claim_Duration

    #Claim Duration = Claim End Date - Claim Start Date
    train_iobp_df['ClaimStartDt'] = pd.to_datetime(train_iobp_df['ClaimStartDt'], format="%Y-%m-%d")
    train_iobp_df['ClaimEndDt'] = pd.to_datetime(train_iobp_df['ClaimEndDt'], format="%Y-%m-%d")

    train_iobp_df['Claim_Duration'] = (train_iobp_df['ClaimEndDt'] - train_iobp_df['ClaimStartDt']).dt.days

    ### **Adding `New Feature - 4` :: `Admitted_Duration`**

    #Admitted Duration = Discharge Date - Admission Date

    train_iobp_df['AdmissionDt'] = pd.to_datetime(train_iobp_df['AdmissionDt'], format="%Y-%m-%d")
    train_iobp_df['DischargeDt'] = pd.to_datetime(train_iobp_df['DischargeDt'], format="%Y-%m-%d")

    train_iobp_df['Admitted_Duration'] = (train_iobp_df['DischargeDt'] - train_iobp_df['AdmissionDt']).dt.days


    ### **Adding `New Feature - 5` :: `Bene_Age`**

    ##Bene Age = DOD - DOB (if DOD is Null then replace it with MAX date in DOD)

    # Filling the Null values as MAX Date of Death in the Dataset
    train_iobp_df['DOD'].fillna(value=train_iobp_df['DOD'].max(), inplace=True)
    train_iobp_df['Bene_Age'] = round(((train_iobp_df['DOD'] - train_iobp_df['DOB']).dt.days)/365,1)

    ### **Adding `New Feature - 6` :: `Att_Opr_Oth_Phy_Tot_Claims`**

    ########     * It represents the total claims submitted by Attending, Operating and Other Physicians.

    #        * **`Reasoning`** :: The idea behind adding this feature is to see whether a total of physicians claims submission will help in influencing the potential frauds.


    #    * **`Logic`** :: Att_Phy_tot_claims + Opr_Phy_tot_claims + Oth_Phy_tot_claims


    train_iobp_df['Att_Phy_tot_claims'] = train_iobp_df.groupby(['AttendingPhysician'])['ClaimID'].transform('count')
    train_iobp_df['Opr_Phy_tot_claims'] = train_iobp_df.groupby(['OperatingPhysician'])['ClaimID'].transform('count')
    train_iobp_df['Oth_Phy_tot_claims'] = train_iobp_df.groupby(['OtherPhysician'])['ClaimID'].transform('count')

    # Creating the combined feature
    train_iobp_df['Att_Phy_tot_claims'].fillna(value=0, inplace=True)
    train_iobp_df['Opr_Phy_tot_claims'].fillna(value=0, inplace=True)
    train_iobp_df['Oth_Phy_tot_claims'].fillna(value=0, inplace=True)

    train_iobp_df['Att_Opr_Oth_Phy_Tot_Claims'] = train_iobp_df['Att_Phy_tot_claims'] + train_iobp_df['Opr_Phy_tot_claims'] + train_iobp_df['Oth_Phy_tot_claims']
    train_iobp_df.drop(['Att_Phy_tot_claims', 'Opr_Phy_tot_claims', 'Oth_Phy_tot_claims'], axis=1, inplace=True)

    ### **Adding `New Feature - 7` :: `Prv_Tot_Att_Opr_Oth_Phys`**

    #    * It represents the total of all kind of physicians that a provider has interacted with.

    #        * **`Reasoning`** :: The idea behind adding this feature is to see whether a fraudulent provider interacts with higher or lower numberof of various physicians.


    #    * **`Logic`** :: Prv_Tot_Att_Phy + Prv_Tot_Opr_Phy + Prv_Tot_Oth_Phy


    train_iobp_df["Prv_Tot_Att_Phy"] = train_iobp_df.groupby(['Provider'])['AttendingPhysician'].transform('count')
    train_iobp_df["Prv_Tot_Opr_Phy"] = train_iobp_df.groupby(['Provider'])['OperatingPhysician'].transform('count')
    train_iobp_df["Prv_Tot_Oth_Phy"] = train_iobp_df.groupby(['Provider'])['OtherPhysician'].transform('count')

    train_iobp_df['Prv_Tot_Att_Opr_Oth_Phys'] = train_iobp_df['Prv_Tot_Att_Phy'] + train_iobp_df['Prv_Tot_Opr_Phy'] + train_iobp_df['Prv_Tot_Oth_Phy']
    train_iobp_df.drop(['Prv_Tot_Att_Phy', 'Prv_Tot_Opr_Phy', 'Prv_Tot_Oth_Phy'], axis=1, inplace=True)


    ### **Adding `New Feature - 8` :: `Total Unique Claim Admit Codes used by a PROVIDER`**

    # * **`Reasoning`** :: The idea behind adding this feature is to see how many unique number of `Claim Admit Diagnosis Codes` used by the Provider. 
    #        * As there may be a pattern that if a provider has used so many Admit Diagnosis Codes then it might increases or decreases the chances of fraud.

    train_iobp_df['PRV_Tot_Admit_DCodes'] = train_iobp_df.groupby(['Provider'])['ClmAdmitDiagnosisCode'].transform('nunique')

    # ### **Adding `New Feature - 9` :: `Total Unique Number of Diagnosis Group Codes used by a PROVIDER`**
    #     * **`Reasoning`** :: The idea behind adding this feature is to see how many unique `Diagnosis Group Codes` used by the Provider.
    #        * As there may be a pattern that if a provider has used so many Diagnosis Group Codes then it might increases or decreases the chances of fraud.

    train_iobp_df['PRV_Tot_DGrpCodes'] = train_iobp_df.groupby(['Provider'])['DiagnosisGroupCode'].transform('nunique')

    ### **Adding `New Feature - 10` :: `Total unique Date of Birth years of beneficiaries provided by a Provider`**

    train_iobp_df['DOB_Year'] = train_iobp_df['DOB'].dt.year
    train_iobp_df['PRV_Tot_Unq_DOB_Years'] = train_iobp_df.groupby(['Provider'])['DOB_Year'].transform('nunique')
    train_iobp_df.drop(['DOB_Year'], axis=1, inplace=True)

    ### **Adding `New Feature - 11` :: `Sum of patients age treated by a Provider`**

    train_iobp_df['PRV_Bene_Age_Sum'] = train_iobp_df.groupby(['Provider'])['Bene_Age'].transform('sum')

    ### **Adding `New Feature - 12` :: `Sum of Insc Claim Re-Imb Amount for a Provider`**

    train_iobp_df['PRV_Insc_Clm_ReImb_Amt'] = train_iobp_df.groupby(['Provider'])['InscClaimAmtReimbursed'].transform('sum')

    ### **Adding `New Feature - 13` :: `Total number of RKD Patients seen by a Provider`**

    train_iobp_df['RenalDiseaseIndicator'] = train_iobp_df['RenalDiseaseIndicator'].apply(lambda val: 1 if val == "Y" else 0)

    train_iobp_df['PRV_Tot_RKD_Patients'] = train_iobp_df.groupby(['Provider'])['RenalDiseaseIndicator'].transform('sum')


    # Dropping these 2 columns as there 99% of values are same
    train_iobp_df.drop(['NoOfMonths_PartACov', 'NoOfMonths_PartBCov'], axis=1, inplace=True)

    # Filling null values in Admitted_Duration with 0 (as it will represent the patients were admitted for 0 days)
    train_iobp_df['Admitted_Duration'].fillna(value=0,inplace=True)
    ### **Adding `Aggregated Features` :: For every possible level**
#     - Provider
#     - Beneficiary
#     - Attending Physician
#     - Operating Physician
#     - Other Physician and etc..
   
   
 # ###  **`Reasoning`** :: The idea behind adding the aggregated features at different levels is that fraud can be done by an individual or group of individuals or entities involved in the claim process.#####''''''

    # PRV Aggregate features
    train_iobp_df["PRV_CoPayment"] = train_iobp_df.groupby('Provider')['DeductibleAmtPaid'].transform('sum')
    train_iobp_df["PRV_IP_Annual_ReImb_Amt"] = train_iobp_df.groupby('Provider')['IPAnnualReimbursementAmt'].transform('sum')
    train_iobp_df["PRV_IP_Annual_Ded_Amt"] = train_iobp_df.groupby('Provider')['IPAnnualDeductibleAmt'].transform('sum')
    train_iobp_df["PRV_OP_Annual_ReImb_Amt"] = train_iobp_df.groupby('Provider')['OPAnnualReimbursementAmt'].transform('sum')
    train_iobp_df["PRV_OP_Annual_Ded_Amt"] = train_iobp_df.groupby('Provider')['OPAnnualDeductibleAmt'].transform('sum')
    train_iobp_df["PRV_Admit_Duration"] = train_iobp_df.groupby('Provider')['Admitted_Duration'].transform('sum')
    train_iobp_df["PRV_Claim_Duration"] = train_iobp_df.groupby('Provider')['Claim_Duration'].transform('sum')    
    #     print(train_iobp_df.shape)
    
        # BENE, PHYs, Diagnosis Admit and Group Codes columns
#     create_agg_feats(grp_col='BeneID', feat_name="BENE")
#     create_agg_feats(grp_col='AttendingPhysician', feat_name="ATT_PHY")
#     create_agg_feats(grp_col='OperatingPhysician', feat_name="OPT_PHY")
#     create_agg_feats(grp_col='OtherPhysician', feat_name="OTH_PHY")
#     create_agg_feats(grp_col='ClmAdmitDiagnosisCode', feat_name="Claim_Admit_Diag_Code")
#     create_agg_feats(grp_col='DiagnosisGroupCode', feat_name="Diag_GCode")
#     # Dropping these 3 columns as there 99% of values are same
#     train_iobp_df.drop(['ClmProcedureCode_4', 'ClmProcedureCode_5', 'ClmProcedureCode_6'], axis=1, inplace=True)
#     # Diagnosis Codes columns
#     create_agg_feats(grp_col='ClmDiagnosisCode_1', feat_name="Claim_DiagCode1")
#     create_agg_feats(grp_col='ClmDiagnosisCode_2', feat_name="Claim_DiagCode2")
#     create_agg_feats(grp_col='ClmDiagnosisCode_3', feat_name="Claim_DiagCode3")
#     create_agg_feats(grp_col='ClmDiagnosisCode_4', feat_name="Claim_DiagCode4")
#     create_agg_feats(grp_col='ClmDiagnosisCode_5', feat_name="Claim_DiagCode5")
#     create_agg_feats(grp_col='ClmDiagnosisCode_6', feat_name="Claim_DiagCode6")
#     create_agg_feats(grp_col='ClmDiagnosisCode_7', feat_name="Claim_DiagCode7")
#     create_agg_feats(grp_col='ClmDiagnosisCode_8', feat_name="Claim_DiagCode8")
#     create_agg_feats(grp_col='ClmDiagnosisCode_9', feat_name="Claim_DiagCode9")
#     create_agg_feats(grp_col='ClmDiagnosisCode_10', feat_name="Claim_DiagCode10")

#     # Medical Procedure Codes columns
#     create_agg_feats(grp_col='ClmProcedureCode_1', feat_name="Claim_ProcCode1")
#     create_agg_feats(grp_col='ClmProcedureCode_2', feat_name="Claim_ProcCode2")
#     create_agg_feats(grp_col='ClmProcedureCode_3', feat_name="Claim_ProcCode3")
    
#     train_iobp_df = Adding_Aggregated_Features()
    
    return train_iobp_df

In [8]:
train_iobp_df = feature_Enginerring(train_bene_df,train_ip_df,train_op_df,train_tgt_lbls_df)

TypeError: 'NoneType' object does not support item assignment

In [ ]:
train_iobp_df.columns

In [ ]:
outputCollection = Helpers.createOutputCollection(context)
out = Helpers.createTemplateOutputDataset(context=context, outputName=outputDataset, dataFrame=train_iobp_df)
outputCollection.addTemplateOutput(out)
Helpers.save(context)